In [ ]:
#importing  all libs

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
files.upload()

In [ ]:
df=pd.read_csv("heart_disease_uci.csv")

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df_clean = df.drop(columns=["id", "dataset"])

In [ ]:
df_clean.fillna(df_clean.median(numeric_only=True), inplace=True)


In [ ]:
categorical_cols = df_clean.select_dtypes(include='object').columns


In [ ]:
for col in categorical_cols:
    df_clean[col].fillna(df_clean[col].mode()[0], inplace=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder

df_clean['target'] = df_clean['num'].apply(lambda x: 1 if x > 0 else 0)
df_clean.drop(columns=['num'], inplace=True)

le = LabelEncoder()
for col in categorical_cols:
    df_clean[col] = le.fit_transform(df_clean[col])
# Final check
df_clean.info(), df_clean.head()


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Split features and target
X = df_clean.drop(columns=['target'])
y = df_clean['target']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Apply SMOTE to balance the training set
#smote = SMOTE(random_state=42)
#X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled.shape, X_test_scaled.shape, y_train.value_counts()


In [ ]:
from sklearn.ensemble import RandomForestClassifier


In [ ]:
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
import matplotlib.pyplot as plt

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)
y_proba_lr = lr.predict_proba(X_test_scaled)[:, 1]
auc_lr = roc_auc_score(y_test, y_proba_lr)

# XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
xgb.fit(X_train_scaled, y_train)
y_pred_xgb = xgb.predict(X_test_scaled)
y_proba_xgb = xgb.predict_proba(X_test_scaled)[:, 1]
auc_xgb = roc_auc_score(y_test, y_proba_xgb)

# LightGBM
lgb = LGBMClassifier(random_state=42)
lgb.fit(X_train_scaled, y_train)
y_pred_lgb = lgb.predict(X_test_scaled)
y_proba_lgb = lgb.predict_proba(X_test_scaled)[:, 1]
auc_lgb = roc_auc_score(y_test, y_proba_lgb)

# Random forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)
y_proba_rf = rf.predict_proba(X_test_scaled)[:, 1]
auc_rf = roc_auc_score(y_test, y_proba_rf)

# ROC Curve
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_proba_rf)



In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_xgb)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_lgb)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred_rf)
print("Accuracy:", accuracy)

In [ ]:
df_clean.head(1)

In [ ]:
new_patient = [[63,	1,	3,	145.0,	233.0,	1,	0,	150.0,	0,	2.3,	0,	0.0,	0]]


In [ ]:

new_patient_scaled = scaler.transform(new_patient)
pred = rf.predict(new_patient_scaled)[0]
if pred == 1:
    print(f"The person likely has heart disease ")
else:
    print(f"The person likely does NOT have heart disease ")
